In [ ]:
#@title Connect/Log-In to Odoo

import xmlrpc.client

# Your Odoo connection details
# We either need the password or the api key
url = 'https://mid-city-engineering.odoo.com/'       # Replace with your Odoo URL
db = 'mid-city-engineering'                          # Your database name
username = 'sales@midcityengineering.com'            # Your login email
api_key = '3eb5c73cc01b4d5279d20a6f727b53dfab0a01f6' # Your API key

common = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/common')
# This API call takes the url but no credential data, so it is public

 # To verify if the connection information is correct before trying to
 # authenticate, the simplest call is to ask for the server’s version.
print('Values returned by common.version():')
print(common.version())
print('\n-----------\n\nDescription of the returned common object:')
type(common)

# Once the url is validated, we use the common.authenticate function to log in

uid = common.authenticate(db, username, api_key, {}) # use api_key or password
print(f"Connected successfully! User ID: {uid}")
models = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/object')

In [ ]:
template = models.execute_kw(db, uid, api_key,
                  'product.template',  # or 'product.template'
                  'search_read',
                   [[
                     ['name', 'ilike', "Rob's Playground"],
                   ]],
                   {
                    'fields': ['name', 'default_code', 'website_url', 'description_ecommerce', 'website_description'],
                   },
                  )
print(f"Template found: {template}")

: 

In [5]:
template[0].get('website_description')

'<section class="s_embed_code pt16 pb16 o_colored_level text-center" data-snippet="s_embed_code" data-name="Embed Code">\n        <template class="s_embed_code_saved"><!-- BEGINNING OF PRODUCT PAGE CUSTOM CONTENT - ADA COMPLIANT\nPASTE THIS INTO ODOO HTML EMBED WIDGET -->\n\n<style>\n    :root {\n        --mce-dark-red: #810305;\n        --mce-sepia-tan: #D4AF85;\n        --mce-black: #0a0a0a;\n        --mce-white: #ffffff;\n    }\n\n    .mce-product-section * {\n        box-sizing: border-box;\n    }\n\n    .mce-product-section {\n        font-family: \'Outfit\', -apple-system, sans-serif;\n        position: relative;\n        min-height: 100vh;\n        background: var(--mce-black);\n        margin-top: 5%;\n        padding-bottom: 60px;\n    }\n\n    /* Screen reader only utility class */\n    .sr-only {\n        position: absolute;\n        width: 1px;\n        height: 1px;\n        padding: 0;\n        margin: -1px;\n        overflow: hidden;\n        clip: rect(0, 0, 0, 0);\n    

In [ ]:
import base64
import os

# Local folder with images
folder_path = './'
files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
print(f"Files found: {files}")

batch_size = 10  # Process in batches to avoid timeouts
for i in range(0, len(files), batch_size):
    batch = files[i:i+batch_size]
    attachments = []
    
    for filename in batch:
        filepath = os.path.join(folder_path, filename)
        with open(filepath, 'rb') as f:
            file_data = base64.b64encode(f.read()).decode('utf-8')
        
        attachment = {
            'name': filename,
            'type': 'binary',
            'datas': file_data,
            'mimetype': 'image/jpeg',  # Adjust based on file
            'res_model': 'documents.document',  # For Documents module
            'public': True  # Makes immediately public
        }
        attachments.append(attachment)
    
    # Bulk create
    models.execute_kw(db, uid, api_key, 'ir.attachment', 'create', [attachments])
    print(f"Uploaded batch {i//batch_size + 1}: {len(batch)} files")


Files found: []


In [11]:
# List all page IDs
page_ids = models.execute_kw(db, uid, api_key, 'website.page', 'search', [[['name', 'ilike', 'Redirect - %']]])


In [12]:
len(page_ids)

111

In [13]:
pages = models.execute_kw(
    db, uid, api_key,
    'website.page', 'read',
    [page_ids],
    {'fields': ['name']}
)


In [19]:
for page in pages[10:]:
    new_name = f"Product {page['name']}"
    models.execute_kw(
        db, uid, api_key,
        'website.page', 'write',
        [[page['id']], {'name': new_name}]
    )


In [24]:
from collections import defaultdict
import re

# Search for all views with names starting with "Product Redirect -"
view_ids = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'search',
    [[['name', 'ilike', 'Product Redirect - SKS']]]
)

# Read the views
views = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'read',
    [view_ids],
    {'fields': ['id', 'name', 'arch_db', 'key']}
)

# Extract redirect URLs
redirect_map = defaultdict(list)
view_redirects = {}

for view in views:
    arch = view['arch_db']
    
    # Extract the redirect URL using regex
    # Looking for window.location.replace('...')
    match = re.search(r"window\.location\.replace\('([^']+)'\)", arch)
    
    if match:
        redirect_url = match.group(1)
        view_info = {
            'id': view['id'],
            'name': view['name'],
            'key': view.get('key', 'N/A')
        }
        
        view_redirects[view['id']] = redirect_url
        redirect_map[redirect_url].append(view_info)

# Print results
print("=" * 80)
print("REDIRECT SUMMARY")
print("=" * 80)
print(f"\nTotal views found: {len(views)}")
print(f"Views with redirects: {len(view_redirects)}")
print(f"Unique redirect URLs: {len(redirect_map)}")

print("\n" + "=" * 80)
print("VIEWS GROUPED BY REDIRECT URL")
print("=" * 80)

for redirect_url, view_list in sorted(redirect_map.items()):
    print(f"\n📍 Redirect to: {redirect_url}")
    print(f"   Number of views: {len(view_list)}")
    
    if len(view_list) > 1:
        print("   ⚠️  MULTIPLE VIEWS redirect to this URL:")
    
    for view_info in view_list:
        print(f"   - ID: {view_info['id']}, Name: {view_info['name']}")
        print(f"     Key: {view_info['key']}")

# Find duplicates
print("\n" + "=" * 80)
print("DUPLICATE REDIRECTS (Multiple views → same URL)")
print("=" * 80)

duplicates = {url: views for url, views in redirect_map.items() if len(views) > 1}

if duplicates:
    for redirect_url, view_list in sorted(duplicates.items()):
        print(f"\n🔄 {len(view_list)} views redirect to: {redirect_url}")
        for view_info in view_list:
            print(f"   - [{view_info['id']}] {view_info['name']}")
else:
    print("\n✅ No duplicates found - each redirect URL is unique")

# Export to CSV (optional)
print("\n" + "=" * 80)
print("CSV Export")
print("=" * 80)

import csv

with open('redirect_views.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['View ID', 'View Name', 'View Key', 'Redirect URL', 'Duplicate Count'])
    
    for view_id, redirect_url in view_redirects.items():
        view = next(v for v in views if v['id'] == view_id)
        duplicate_count = len(redirect_map[redirect_url])
        writer.writerow([
            view['id'],
            view['name'],
            view.get('key', 'N/A'),
            redirect_url,
            duplicate_count
        ])

print("✅ Exported to redirect_views.csv")

REDIRECT SUMMARY

Total views found: 26
Views with redirects: 26
Unique redirect URLs: 26

VIEWS GROUPED BY REDIRECT URL

📍 Redirect to: /shop/product-smartkey-starter-remote-start-alarm-for-mercedes-benz-freightliner-sprinter-sksng907rv-9791
   Number of views: 1
   - ID: 4974, Name: Product Redirect - SKSNG907RV
     Key: website.product-redirect-sksng907rv

📍 Redirect to: /shop/sks906-smrt-b-sks906-smrt-b-9744
   Number of views: 1
   - ID: 4977, Name: Product Redirect - SKS906-SMRT-B
     Key: website.product-redirect-sks906-smrt-b

📍 Redirect to: /shop/sksng164-smartkey-starter-r-for-mercedes-w164-x164-9762
   Number of views: 1
   - ID: 4985, Name: Product Redirect - SKSNG164
     Key: website.product-redirect-sksng164

📍 Redirect to: /shop/sksng166d3-sksng166d3-smartkey-starter-r-remote-start-for-mercedes-das3-9764
   Number of views: 1
   - ID: 4966, Name: Product Redirect - SKSNG166D3
     Key: website.product-redirect-sksng166d3

📍 Redirect to: /shop/sksng166d4-sksng166d4-sma

In [22]:
!ls

redirect_views.csv  sample_data


In [23]:
!cat redirect_views.csv

View ID,View Name,View Key,Redirect URL,Duplicate Count
5042,Product Redirect - 205INTK,website.product-redirect-205intk,/shop/205intk-plug-play-alarm-kit-for-mercedes-w205-9641,1
5046,Product Redirect - 9064CLOBD,website.product-redirect-9064clobd,/shop/9064clobd-906-v4-obd-high-idle-programmer-9120,1
4950,Product Redirect - 906CAM,website.product-redirect-906cam,/shop/906cam-w906-sprinter-van-rear-camera-obd-programmer-10359,1
4951,Product Redirect - 906CAMKLIGHT,website.product-redirect-906camklight,/shop/906camklight-906camklight-9125,1
4993,Product Redirect - 906PRE,website.product-redirect-906pre,/shop/906pre-camera-adapter-for-oem-pre-wiring-w906-9128,1
5053,Product Redirect - 907ALARMDIS,website.product-redirect-907alarmdis,/shop/907alarmdis-oem-alarm-disable-obd-programmer-for-907-sprinter-9615,1
5019,Product Redirect - 907ANT,website.product-redirect-907ant,/shop/907ant-oem-am-fm-antenna-adapter-for-w907-sprinter-9136,1
5018,Product Redirect - 907CAM,website.product-redirect-

In [26]:
view_ids = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'search',
    [[['name', 'ilike', 'Product Redirect - SKS%']]]
)

# Read the views
views = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'read',
    [view_ids],
    {'fields': ['id', 'name', 'arch_db', 'key']}
)

# Extract and display redirect URLs
print("=" * 80)
print("PRODUCT REDIRECT - SKS VIEWS")
print("=" * 80)
print(f"\nTotal views found: {len(views)}\n")

redirects = []

for view in views:
    arch = view['arch_db']
    
    # Extract the redirect URL using regex
    match = re.search(r"window\.location\.replace\('([^']+)'\)", arch)
    
    if match:
        redirect_url = match.group(1)
        redirects.append({
            'id': view['id'],
            'name': view['name'],
            'key': view.get('key', 'N/A'),
            'redirect': redirect_url
        })
        
        print(f"View ID: {view['id']}")
        print(f"Name: {view['name']}")
        print(f"Key: {view.get('key', 'N/A')}")
        print(f"➜ Redirects to: {redirect_url}")
        print("-" * 80)
    else:
        print(f"⚠️  View ID: {view['id']}")
        print(f"Name: {view['name']}")
        print(f"❌ No redirect found in architecture")
        print("-" * 80)

# Summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total SKS redirect views: {len(views)}")
print(f"Views with valid redirects: {len(redirects)}")
print(f"Views without redirects: {len(views) - len(redirects)}")

# Quick list format
if redirects:
    print("\n" + "=" * 80)
    print("QUICK REFERENCE LIST")
    print("=" * 80)
    for item in redirects:
        print(f"{item['name']}")
        print(f"  → {item['redirect']}")
        print()

# Export to text file for easy reference
with open('sks_redirects.txt', 'w', encoding='utf-8') as f:
    f.write("SKS Product Redirects\n")
    f.write("=" * 80 + "\n\n")
    
    for item in redirects:
        f.write(f"View: {item['name']}\n")
        f.write(f"Redirect: {item['redirect']}\n")
        f.write("\n")

print("\n✅ Exported to sks_redirects.txt")


PRODUCT REDIRECT - SKS VIEWS

Total views found: 26

View ID: 4977
Name: Product Redirect - SKS906-SMRT-B
Key: website.product-redirect-sks906-smrt-b
➜ Redirects to: /shop/sks906-smrt-b-sks906-smrt-b-9744
--------------------------------------------------------------------------------
View ID: 4985
Name: Product Redirect - SKSNG164
Key: website.product-redirect-sksng164
➜ Redirects to: /shop/sksng164-smartkey-starter-r-for-mercedes-w164-x164-9762
--------------------------------------------------------------------------------
View ID: 4966
Name: Product Redirect - SKSNG166D3
Key: website.product-redirect-sksng166d3
➜ Redirects to: /shop/sksng166d3-sksng166d3-smartkey-starter-r-remote-start-for-mercedes-das3-9764
--------------------------------------------------------------------------------
View ID: 4964
Name: Product Redirect - SKSNG166D4
Key: website.product-redirect-sksng166d4
➜ Redirects to: /shop/sksng166d4-sksng166d4-smartkey-starter-r-for-mercedes-das4-with-kg-9765
------------

In [27]:
!cat sks_redirects.txt

SKS Product Redirects

View: Product Redirect - SKS906-SMRT-B
Redirect: /shop/sks906-smrt-b-sks906-smrt-b-9744

View: Product Redirect - SKSNG164
Redirect: /shop/sksng164-smartkey-starter-r-for-mercedes-w164-x164-9762

View: Product Redirect - SKSNG166D3
Redirect: /shop/sksng166d3-sksng166d3-smartkey-starter-r-remote-start-for-mercedes-das3-9764

View: Product Redirect - SKSNG166D4
Redirect: /shop/sksng166d4-sksng166d4-smartkey-starter-r-for-mercedes-das4-with-kg-9765

View: Product Redirect - SKSNG166D4T
Redirect: /shop/sksng166d4t-sksng166d4t-smartkey-starter-r-for-mercedes-non-kg-das4-9766

View: Product Redirect - SKSNG167
Redirect: /shop/sksng167-smartkey-starter-r-for-mercedes-w167-and-w463-9767

View: Product Redirect - SKSNG190D4
Redirect: /shop/sksng190d4-sksng190d4-9768

View: Product Redirect - SKSNG197D3
Redirect: /shop/sksng197d3-sksng197d3-9769

View: Product Redirect - SKSNG204D3
Redirect: /shop/sksng204d3-smartkey-starter-r-remote-start-for-mercedes-sksng204d3-9771

Vie

In [30]:
import re

# Search for all views with names starting with "Product Redirect - SKS"
view_ids = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'search',
    [[['name', 'ilike', 'Product Redirect - SKS%']]]
)

# Read the views
views = models.execute_kw(db, uid, api_key,
    'ir.ui.view', 'read',
    [view_ids],
    {'fields': ['id', 'name', 'arch_db']}
)

# Extract product template IDs
product_template_ids = []

for view in views:
    arch = view['arch_db']
    
    # Extract the redirect URL
    match = re.search(r"window\.location\.replace\('([^']+)'\)", arch)
    
    if match:
        redirect_url = match.group(1)
        
        # Extract the product template ID (number after the last dash)
        id_match = re.search(r'-(\d+)$', redirect_url)
        
        if id_match:
            product_template_id = int(id_match.group(1))
            product_template_ids.append(product_template_id)

print(f"Found {len(product_template_ids)} product template IDs")
print(product_template_ids)

Found 26 product template IDs
[9744, 9762, 9764, 9765, 9766, 9767, 9768, 9769, 9771, 9772, 9773, 9776, 9777, 9778, 9779, 9781, 9782, 9784, 9785, 9786, 9787, 9788, 9790, 9791, 9797, 9811]


In [31]:
duplicates = [id for id in set(product_template_ids) if product_template_ids.count(id) > 1]

print(f"Duplicates: {duplicates}")
print(f"Number of duplicates: {len(duplicates)}")

Duplicates: []
Number of duplicates: 0


In [32]:
# Fetch product names
products = models.execute_kw(db, uid, api_key,
    'product.template', 'read',
    [product_template_ids],
    {'fields': ['id', 'name']}
)

for product in products:
    print(f"ID {product['id']}: {product['name']}")

ID 9744: SKS906-SMRT-B
ID 9762: SmartKey Starter® for Mercedes W164/X164
ID 9764: SKSNG166D3 SmartKey Starter® Remote Start for Mercedes DAS3
ID 9765: SKSNG166D4 SmartKey Starter® for Mercedes DAS4 with KG
ID 9766: SKSNG166D4T SmartKey Starter® for Mercedes Non-KG DAS4
ID 9767: SmartKey Starter® for Mercedes W167 and W463
ID 9768: SKSNG190D4
ID 9769: SKSNG197D3
ID 9771: SmartKey Starter® Remote Start for Mercedes – SKSNG204D3
ID 9772: SKSNG204D4 SmartKey Starter® for Mercedes DAS4 with KG
ID 9773: SKSNG204D4T SmartKey Starter® for Mercedes DAS4 Non-KG
ID 9776: SmartKey Starter® for Mercedes W205/X205 DAS4 with KG
ID 9777: SKSNG211 SmartKey Starter® for Mercedes W203, W211, C209, W219, R171, W463 
ID 9778: SmartKey Starter® for Mercedes SKSNG213
ID 9779: SmartKey Starter® for Mercedes W220, W230
ID 9781: SmartKey Starter® Remote Start for W221, W218 Mercedes
ID 9782: SmartKey Starter® for Mercedes S Class W222
ID 9784: SKSNG231D4
ID 9785: SmartKey Starter® for Mercedes Metris W447
ID 97

In [49]:
# Fetch product names
products = models.execute_kw(db, uid, api_key,
    'product.template', 'read',
    [9771],
    {}
)

for product in products:
    print(f"ID {product['id']}: {product['name']}")
    # print(f"Description: {product['description_ecommerce']}")
    # print(f"Website Description: {product['website_description']}")

ID 9771: SmartKey Starter® Remote Start for Mercedes – SKSNG204D3


In [46]:
if '9771' in product['website_description']:
    print("found")
else:
    print("not found")

not found


In [51]:
new_website_description = product['website_description']
new_description_ecommerce = product['description_ecommerce']

In [54]:
# Update products in batches with error handling
import time

batch_size = 5
successful = 0
failed = []

for i in range(0, len(product_template_ids), batch_size):
    batch = product_template_ids[i:i+batch_size]
    
    try:
        models.execute_kw(db, uid, api_key,
            'product.template', 'write',
            [batch, {
                'description_ecommerce': new_description_ecommerce,
                'website_description': new_website_description,
                'is_published': True
            }]
        )
        successful += len(batch)
        print(f"Updated batch {i//batch_size + 1}: {len(batch)} products")
        time.sleep(0.5)  # Small delay between batches
        
    except Exception as e:
        print(f"Error updating batch {i//batch_size + 1}: {e}")
        failed.extend(batch)

print(f"\nTotal successful: {successful}")
print(f"Total failed: {len(failed)}")
if failed:
    print(f"Failed IDs: {failed}")

Updated batch 1: 5 products
Updated batch 2: 5 products
Updated batch 3: 5 products
Updated batch 4: 5 products
Updated batch 5: 5 products
Updated batch 6: 1 products

Total successful: 26
Total failed: 0
